In [1]:
import statsmodels.formula.api as smf
import fastreg as fr
from fastreg import I, R, C, C0

### Generate Data

In [2]:
models = ['linear', 'poisson', 'logit', 'negbin']
data = fr.dataset(N=1_000_000, K1=10, K2=100, models=models, seed=89320432)
data_wide = fr.dataset(N=1_000_000, K1=10, K2=10_000, models=models, seed=89320433)
data.head()

,x1,x2,yhat0,yhat,id1,id2,y0,y,Eb0,Eb,b0,b,Ep0,Ep,p0,p,nb0,nb
0,-1.429225,1.830295,0.769410,2.149410,H,68,1.791671,1.585510,0.683393,0.895614,1,1,2.158492,8.579792,4,10,1,5
1,0.687153,0.795373,0.783369,1.403369,C,42,0.295552,3.163529,0.686406,0.802718,1,1,2.188835,4.068886,1,2,1,0
2,0.764353,-1.060225,-0.306829,-0.036829,C,7,0.381637,-0.639362,0.423889,0.490794,1,1,0.735777,0.963841,0,1,2,0
3,-0.309887,-0.777701,-0.459587,0.800413,D,96,0.143218,1.139854,0.387084,0.690063,0,1,0.631545,2.226461,0,0,1,3
4,-0.616042,-0.752266,-0.536172,0.623828,H,46,-0.468395,1.462211,0.369078,0.651089,1,1,0.584983,1.866057,0,1,1,7


In [3]:
# for statsmodels runs
data1 = data.copy()
data1['id2'] = data1['id2'].astype(str)

### Normal OLS

In [4]:
%time fr.ols(y=R.y0, x=I+R.x1+R.x2, data=data)

CPU times: user 42.6 ms, sys: 77 ms, total: 120 ms
Wall time: 21.5 ms


y0,coeff,stderr,low95,high95,pvalue
I,0.101859,0.001000,0.099900,0.103818,0.0
x1,0.301647,0.001000,0.299688,0.303606,0.0
x2,0.599408,0.000999,0.597450,0.601366,0.0


In [5]:
%time smf.ols('y0 ~ 1 + x1 + x2', data=data).fit().params

CPU times: user 373 ms, sys: 1.22 s, total: 1.6 s
Wall time: 129 ms


Intercept    0.101859
x1           0.301647
x2           0.599408
dtype: float64

In [6]:
%time fr.ols(y=R.y, x=I+R.x1+R.x2+C.id1+C.id2, data=data)

CPU times: user 434 ms, sys: 767 ms, total: 1.2 s
Wall time: 316 ms


y,coeff,stderr,low95,high95,pvalue
I,0.112463,0.010413,0.092054,0.132872,0.0
x1,0.299557,0.001001,0.297595,0.301518,0.0
x2,0.601840,0.001000,0.599880,0.603800,0.0
id1=B,0.104734,0.004476,0.095962,0.113507,0.0
id1=C,0.201965,0.004474,0.193197,0.210734,0.0
...,...,...,...,...,...
id2=95,0.937461,0.014102,0.909821,0.965101,0.0
id2=96,0.956624,0.014109,0.928970,0.984278,0.0
id2=97,0.960951,0.014099,0.933318,0.988584,0.0
id2=98,0.993398,0.014132,0.965699,1.021097,0.0


In [7]:
%time smf.ols('y ~ 1 + x1 + x2 + id1 + id2', data=data1).fit().params

CPU times: user 1min 37s, sys: 16.3 s, total: 1min 53s
Wall time: 10.8 s


Intercept    0.112463
id1[T.B]     0.104734
id1[T.C]     0.201965
id1[T.D]     0.300212
id1[T.E]     0.403094
               ...   
id2[T.97]    0.960951
id2[T.98]    0.993398
id2[T.99]    0.988670
x1           0.299557
x2           0.601840
Length: 111, dtype: float64

### High Dimensional

In [8]:
%time fr.ols(y=R.y, x=I+R.x1+R.x2+C.id1+C.id2, data=data_wide)

CPU times: user 4.05 s, sys: 2.08 s, total: 6.13 s
Wall time: 4.45 s


y,coeff,stderr,low95,high95,pvalue
I,0.143877,0.103699,-0.059369,0.347123,1.653041e-01
x1,0.301630,0.001004,0.299662,0.303597,0.000000e+00
x2,0.599284,0.001006,0.597313,0.601255,0.000000e+00
id1=B,0.097642,0.004499,0.088824,0.106461,0.000000e+00
id1=C,0.189008,0.004491,0.180207,0.197810,0.000000e+00
...,...,...,...,...,...
id2=9995,1.045182,0.146196,0.758643,1.331722,8.730794e-13
id2=9996,1.029893,0.146587,0.742588,1.317198,2.128075e-12
id2=9997,0.892911,0.142021,0.614554,1.171267,3.233234e-10
id2=9998,0.961372,0.144705,0.677755,1.244989,3.059886e-11


In [9]:
%time fr.ols(y=R.y, x=I+R.x1+R.x2+C.id1, hdfe=C.id2, data=data_wide)

CPU times: user 368 ms, sys: 42.8 ms, total: 411 ms
Wall time: 362 ms


y,coeff,stderr,low95,high95,pvalue
I,0.143877,0.103699,-0.059369,0.347123,1.653041e-01
x1,0.301630,0.001004,0.299662,0.303597,0.000000e+00
x2,0.599284,0.001006,0.597313,0.601255,0.000000e+00
id1=B,0.097642,0.004499,0.088824,0.106461,0.000000e+00
id1=C,0.189008,0.004491,0.180207,0.197810,0.000000e+00
...,...,...,...,...,...
id2=9995,1.045182,0.146196,0.758643,1.331722,8.730794e-13
id2=9996,1.029893,0.146587,0.742588,1.317198,2.128075e-12
id2=9997,0.892911,0.142021,0.614554,1.171267,3.233234e-10
id2=9998,0.961372,0.144705,0.677755,1.244989,3.059886e-11


In [10]:
%time fr.ols(y=R.y, x=I+R.x1+R.x2+C.id1, absorb=C.id2, data=data_wide)

CPU times: user 791 ms, sys: 1.86 s, total: 2.65 s
Wall time: 371 ms


y,coeff,stderr,low95,high95,pvalue
I,0.605524,0.003035,0.599576,0.611471,0.0
x1,0.301630,0.000998,0.299674,0.303585,0.0
x2,0.599284,0.001014,0.597297,0.601270,0.0
id1=B,0.097642,0.004531,0.088762,0.106523,0.0
id1=C,0.189008,0.004545,0.180100,0.197917,0.0
id1=D,0.295160,0.004506,0.286328,0.303991,0.0
id1=E,0.402802,0.004510,0.393963,0.411641,0.0
id1=F,0.495409,0.004493,0.486602,0.504215,0.0
id1=G,0.596910,0.004467,0.588155,0.605665,0.0
id1=H,0.694326,0.004490,0.685526,0.703125,0.0


### Poisson

In [11]:
%time fr.poisson(y=R.p0, x=I+R.x1+R.x2, data=data)

[  0] ℓ=-0.63803, g=0.02960, Δβ=0.60303, Δℓ=inf, μR=0.33633, μC=nan
[ 31] ℓ=-0.62216, g=0.00010, Δβ=0.00000, Δℓ=0.00000, μR=0.33319, μC=nan
CPU times: user 5.9 s, sys: 1.17 s, total: 7.06 s
Wall time: 4.84 s


p0,coeff,stderr,low95,high95,pvalue
I,0.097462,0.001023,0.095458,0.099467,0.0
x1,0.301945,0.000849,0.300281,0.303609,0.0
x2,0.600163,0.000848,0.598502,0.601824,0.0


In [12]:
%time fr.poisson(y=R.p, x=I+R.x1+R.x2+C.id1+C.id2, data=data)

[  0] ℓ=2.39265, g=0.08783, Δβ=0.84455, Δℓ=inf, μR=0.41699, μC=0.37467
[ 31] ℓ=2.55489, g=0.00012, Δβ=0.00008, Δℓ=0.00000, μR=0.33582, μC=0.49825
CPU times: user 10.2 s, sys: 2.49 s, total: 12.7 s
Wall time: 7 s


p,coeff,stderr,low95,high95,pvalue
I,0.104049,0.006941,0.090444,0.117653,0.0
x1,0.303011,0.000508,0.302016,0.304007,0.0
x2,0.600389,0.000506,0.599397,0.601381,0.0
id1=B,0.094683,0.002832,0.089132,0.100234,0.0
id1=C,0.206832,0.002770,0.201402,0.212262,0.0
...,...,...,...,...,...
id2=95,0.949977,0.007787,0.934714,0.965239,0.0
id2=96,0.952913,0.007813,0.937599,0.968227,0.0
id2=97,0.967282,0.007822,0.951952,0.982612,0.0
id2=98,0.997022,0.007779,0.981776,1.012268,0.0


### Logit

In [13]:
%time fr.logit(y=R.b0, x=I+R.x1+R.x2, data=data)

[  0] ℓ=-0.64539, g=0.00459, Δβ=0.59981, Δℓ=inf, μR=0.34471, μC=nan
[ 31] ℓ=-0.64337, g=0.00007, Δβ=0.00000, Δℓ=0.00000, μR=0.33704, μC=nan
CPU times: user 4.96 s, sys: 1.12 s, total: 6.08 s
Wall time: 3.57 s


b0,coeff,stderr,low95,high95,pvalue
I,0.101587,0.002104,0.097463,0.105711,0.0
x1,0.304348,0.002146,0.300143,0.308554,0.0
x2,0.605197,0.002269,0.600749,0.609644,0.0


In [14]:
%time fr.logit(y=R.b, x=I+R.x1+R.x2+C.id1+C.id2, data=data)

[  0] ℓ=-0.55010, g=0.00373, Δβ=0.91517, Δℓ=inf, μR=0.40452, μC=0.39485
[ 40] ℓ=-0.54383, g=0.00010, Δβ=0.00017, Δℓ=0.00000, μR=0.35234, μC=0.47504
[ 73] ℓ=-0.54383, g=0.00010, Δβ=0.00010, Δℓ=0.00000, μR=0.35092, μC=0.47720
CPU times: user 18.4 s, sys: 3.85 s, total: 22.2 s
Wall time: 12.7 s


b,coeff,stderr,low95,high95,pvalue
I,0.146217,0.022841,0.101449,0.190985,1.539278e-10
x1,0.312353,0.002382,0.307685,0.317021,0.000000e+00
x2,0.594196,0.002508,0.589280,0.599111,0.000000e+00
id1=B,0.081497,0.009835,0.062222,0.100773,2.220446e-16
id1=C,0.202393,0.009910,0.182971,0.221816,0.000000e+00
...,...,...,...,...,...
id2=95,0.838382,0.033492,0.772740,0.904024,0.000000e+00
id2=96,0.955043,0.034054,0.888298,1.021788,0.000000e+00
id2=97,0.916935,0.033867,0.850556,0.983314,0.000000e+00
id2=98,0.919533,0.034025,0.852845,0.986220,0.000000e+00


### Ultra Wide

In [15]:
N = 5_000_000
df = pd.DataFrame({ 
    'x1': np.random.rand(N), 
    'x2': np.random.rand(N), 
    'id1': np.ceil(10*np.arange(N)/N+1e-7).astype(int),
    'id2': np.random.randint(1, 100_001, size=N)
})
df['y'] = 1 + 2*df['x1'] + 3*df['x2'] + np.log10(df['id1']) + np.log10(df['id2']) + np.random.randn(N)
print(df[['id1', 'id2']].nunique())

id1        10
id2    100000
dtype: int64


In [16]:
%time fr.ols(y=R.y, x=I+R.x1+R.x2+C.id1, hdfe=C.id2, data=df)

CPU times: user 2.03 s, sys: 638 ms, total: 2.67 s
Wall time: 2 s


y,coeff,stderr,low95,high95,pvalue
I,1.102114,0.142859,0.822115,1.382114,1.221245e-14
x1,2.000176,0.001565,1.997108,2.003243,0.000000e+00
x2,2.999761,0.001564,2.996695,3.002828,0.000000e+00
id1=2,0.299173,0.002020,0.295214,0.303132,0.000000e+00
id1=3,0.475625,0.002020,0.471665,0.479584,0.000000e+00
...,...,...,...,...,...
id2=99996,5.010957,0.199081,4.620765,5.401149,0.000000e+00
id2=99997,4.709422,0.205283,4.307075,5.111770,0.000000e+00
id2=99998,4.664567,0.190463,4.291267,5.037867,0.000000e+00
id2=99999,4.969195,0.214575,4.548635,5.389755,0.000000e+00
